In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from datetime import datetime
import dill
import requests
from bs4 import BeautifulSoup
import html
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from collections import defaultdict
import json

Two sources of data:
- Before 01-08-2021, Trump’s twitter archive, https://www.thetrumparchive.com/  
- After 01-08-2021, the Gab website, https://gab.com/realdonaldtrump, after Trump's Twitter account was suspended. 

In [2]:
# File to load, a csv file available for downloading from Trump’s twitter archive, https://www.thetrumparchive.com/

tweets_data_to_load = "data/tweets_01-08-2021.csv"

In [3]:
# Read the Trump's tweets data prior to or on 01-08-2021
trump_tweets1 = pd.read_csv(tweets_data_to_load)
trump_tweets1

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,98454970654916608,Republicans and Democrats have both created ou...,f,f,TweetDeck,49,255,2011-08-02 18:07:48,f
1,1234653427789070336,I was thrilled to be back in the Great city of...,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,f
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance...,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47,f
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,f
4,1218159531554897920,RT @MZHemingway: Very friendly telling of even...,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59,f
...,...,...,...,...,...,...,...,...,...
56566,1319485303363571714,RT @RandPaul: I don’t know why @JoeBiden think...,t,f,Twitter for iPhone,0,20683,2020-10-23 03:46:25,f
56567,1319484210101379072,RT @EliseStefanik: President @realDonaldTrump ...,t,f,Twitter for iPhone,0,9869,2020-10-23 03:42:05,f
56568,1319444420861829121,RT @TeamTrump: LIVE: Presidential Debate #Deba...,t,f,Twitter for iPhone,0,8197,2020-10-23 01:03:58,f
56569,1319384118849949702,Just signed an order to support the workers of...,f,f,Twitter for iPhone,176289,36001,2020-10-22 21:04:21,f


In [4]:
# Create a dataframe consisting of selected columns
trump_tweets12 = trump_tweets1.loc[:, ["text", "date", "favorites"]]
trump_tweets12

,text,date,favorites
0,Republicans and Democrats have both created ou...,2011-08-02 18:07:48,49
1,I was thrilled to be back in the Great city of...,2020-03-03 01:34:50,73748
2,RT @CBS_Herridge: READ: Letter to surveillance...,2020-01-17 03:22:47,0
3,The Unsolicited Mail In Ballot Scam is a major...,2020-09-12 20:10:58,80527
4,RT @MZHemingway: Very friendly telling of even...,2020-01-17 13:13:59,0
...,...,...,...
56566,RT @RandPaul: I don’t know why @JoeBiden think...,2020-10-23 03:46:25,0
56567,RT @EliseStefanik: President @realDonaldTrump ...,2020-10-23 03:42:05,0
56568,RT @TeamTrump: LIVE: Presidential Debate #Deba...,2020-10-23 01:03:58,0
56569,Just signed an order to support the workers of...,2020-10-22 21:04:21,176289


In [5]:
# Trump's tweets Data after 01-08-2021, api called from the Gab website, https://gab.com/realdonaldtrump
result2= !garc userposts realdonaldtrump --gabs_after=2021-01-08

In [6]:
list=[]
for gab in result2:
    list.append(json.loads(gab))
trump_tweets2 = pd.DataFrame(list)
trump_tweets2

,id,created_at,revised_at,in_reply_to_id,in_reply_to_account_id,sensitive,spoiler_text,visibility,language,uri,...,quote,account,group,media_attachments,mentions,tags,emojis,card,poll,body
0,106371747351370740,2021-06-07T22:14:15.057Z,None,None,None,False,,public,en,/realdonaldtrump/posts/106371747351370740,...,None,"{'id': '311', 'username': 'realdonaldtrump', '...",None,[],[],[],[],None,None,Why does Fox News keep Chris Wallace? His rati...
1,106371746793252577,2021-06-07T22:14:06.550Z,None,None,None,False,,public,en,/realdonaldtrump/posts/106371746793252577,...,None,"{'id': '311', 'username': 'realdonaldtrump', '...",None,[],[],[],[],None,None,Senator Lisa Murkowski has cost the great peop...
2,106369793102394551,2021-06-07T13:57:15.591Z,None,None,None,False,,public,None,/realdonaldtrump/posts/106369793102394551,...,None,"{'id': '311', 'username': 'realdonaldtrump', '...",None,"[{'id': '76126494', 'type': 'video', 'url': 'h...",[],[],[],None,None,
3,106353944549143965,2021-06-04T18:46:45.863Z,None,None,None,False,,public,en,/realdonaldtrump/posts/106353944549143965,...,None,"{'id': '311', 'username': 'realdonaldtrump', '...",None,[],[],[],[],None,None,Next time I’m in the White House there will be...
4,106353621786364303,2021-06-04T17:24:40.885Z,None,None,None,False,,public,en,/realdonaldtrump/posts/106353621786364303,...,None,"{'id': '311', 'username': 'realdonaldtrump', '...",None,[],[],[],[],None,None,Facebook’s ruling is an insult to the record-s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,105520641072854264,2021-01-08T14:46:43.965Z,None,None,None,False,,public,en,/realdonaldtrump/posts/105520641072854264,...,None,"{'id': '311', 'username': 'realdonaldtrump', '...",None,[],[],[],[],None,None,"The 75,000,000 great American Patriots who vot..."
156,105517255346000455,2021-01-08T00:25:41.792Z,None,None,None,False,,public,None,/realdonaldtrump/posts/105517255346000455,...,None,"{'id': '311', 'username': 'realdonaldtrump', '...",None,[],[],[],[],"{'id': 8206540, 'url': 'https://tv.gab.com/cha...",None,https://tv.gab.com/channel/realdonaldtrump/vie...
157,105510926218134347,2021-01-06T21:36:06.987Z,None,None,None,False,,public,None,/realdonaldtrump/posts/105510926218134347,...,None,"{'id': '311', 'username': 'realdonaldtrump', '...",None,[],[],[],[],"{'id': 8181514, 'url': 'https://tv.gab.com/cha...",None,https://tv.gab.com/channel/realdonaldtrump/vie...
158,105510632079096592,2021-01-06T20:21:19.003Z,None,None,None,False,,public,en,/realdonaldtrump/posts/105510632079096592,...,None,"{'id': '311', 'username': 'realdonaldtrump', '...",None,[],[],[],[],None,None,I am asking for everyone at the U.S. Capitol t...


In [7]:
# Clean the trump_tweets2 data and create a dataframe consisting of the same selected columns similar to trump_tweet12  
trump_tweets2['created_at'] = trump_tweets2['created_at'].str.replace('T', ' ').str.replace('Z', '')
trump_tweets2['created_at'] = trump_tweets2['created_at'] .astype(str)
trump_tweets2['created_at'] = trump_tweets2['created_at'] .str.replace('\..*','')
trump_tweets22 = trump_tweets2.loc[:, ["body", "created_at", "favourites_count"]]
trump_tweets22.rename(columns={"body": "text", "created_at": "date", "favourites_count": "favorites"}, inplace=True)
trump_tweets22

,text,date,favorites
0,Why does Fox News keep Chris Wallace? His rati...,2021-06-07 22:14:15,1216
1,Senator Lisa Murkowski has cost the great peop...,2021-06-07 22:14:06,1171
2,,2021-06-07 13:57:15,2483
3,Next time I’m in the White House there will be...,2021-06-04 18:46:45,7720
4,Facebook’s ruling is an insult to the record-s...,2021-06-04 17:24:40,8062
...,...,...,...
155,"The 75,000,000 great American Patriots who vot...",2021-01-08 14:46:43,104325
156,https://tv.gab.com/channel/realdonaldtrump/vie...,2021-01-08 00:25:41,54075
157,https://tv.gab.com/channel/realdonaldtrump/vie...,2021-01-06 21:36:06,43443
158,I am asking for everyone at the U.S. Capitol t...,2021-01-06 20:21:19,41256


- Vertical merge the above two dataframes, trump_tweets12 and trump_tweets22, for a trump_tweets dataframe as a preparation for nlp analysis.
- Sort the dataframe in descending date order for a purpose of web display which allows about 125,000 rows so that the most recent rows meet the cut.    

In [8]:
frames = [trump_tweets12, trump_tweets22]
trump_tweets = pd.concat(frames)
trump_tweets['date'] = pd.to_datetime(trump_tweets.date)
trump_tweets.sort_values('date', inplace=True)
trump_tweets = trump_tweets.loc[trump_tweets['date']>'2016-01-01']
trump_tweets = trump_tweets.loc[:, ["date", "text", "favorites" ]].dropna()
trump_tweets.sort_values(by=['date'], inplace=True, ascending=False)

trump_tweets.head()

,date,text,favorites
0,2021-06-07 22:14:15,Why does Fox News keep Chris Wallace? His rati...,1216
1,2021-06-07 22:14:06,Senator Lisa Murkowski has cost the great peop...,1171
2,2021-06-07 13:57:15,,2483
3,2021-06-04 18:46:45,Next time I’m in the White House there will be...,7720
4,2021-06-04 17:24:40,Facebook’s ruling is an insult to the record-s...,8062


In [9]:
# Save the dataframe to a csv file for web display usage
trump_tweets.to_csv('./Data/trump_tweeters.csv', index=False)

In [10]:
tweets = trump_tweets.text

In [11]:
# Instantiate Count Vectorizer

my_additional_stop_words = ['https','rt']

stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)
vectorizer = CountVectorizer( lowercase=True, stop_words=stop_words)

vector_text = tweets

# To actually create the vectorizer, we simply need to call fit on the text data that we wish to fix
cv_fit = vectorizer.fit_transform(vector_text)

In [12]:
words = vectorizer.vocabulary_

In [13]:
feature_names = vectorizer.get_feature_names()
feature_counts = cv_fit.toarray().sum(axis=0)

In [14]:
final_count_df = pd.DataFrame({"word": feature_names, "cnt_word": feature_counts})
final_count_df.sort_values(by='cnt_word', ascending=False)

,word,cnt_word
13563,great,4543
3590,amp,3742
23875,president,3173
25164,realdonaldtrump,3054
30789,trump,2908
...,...,...
14942,hvyjrbiycc,1
14941,hvvhsmbg7s,1
14940,hvvfb5cprg,1
14939,hvrzcet0tr,1


In [15]:
final_count_df.to_csv('./Data/trump_tw_count.csv', index=False)

In [16]:
# Introduce adjusted economic keywords extracted from top 100 words in the combolist in Powell's speeches + keywords in economic indicators
trade_matchers = ["bankruptcies",
"broadband internet penetration",
"building permit",
"business cycle",
"central banks",
"changes in",
"china",
"construction spending",
"consumer confidence index",
"consumer leverage ratio",
"consumer price index",
"consumer spending",
"during the",
"economic growth",
"employment",
"export",
"federal funds",
"federal open",
"federal reserve",
"financial crisis",
"financial stability",
"financial system",
"forward to",
"funds rate",
"gdp",
"home building",
"housing starts",
"import",
"importance of",
"industrial production",
"inflation",
"interest rate spread",
"interest rates",
"jobless claims",
"labor market",
"level of",
"likely to",
"longer term",
"look forward",
"manufacture new order",
"manufacture",
"manufacturing demand",
"manufacturing",
"market committee",
"market",
"maximum employment",
"monetary policy",
"money supply",
"money",
"monthly job added",
"NAFTA",
"NASDAQ",
"need to",
"new residential sales",
"oil",
"open market",
"opportunity to",
"our percent",
"over time",
"percent objective",
"prime rate",
"producer price index",
"quarterly change in gdp",
"retail sales",
"review of",
"S&P 500",
"short term",
"stock market prices",
"stock market",
"stocks",
"tariff",
"tax",
"the committee",
"the economy",
"the fed",
"the federal",
"the fomc",
"the great",
"the outlook",
"the united",
"the world",
"to address",
"to continue",
"to ensure",
"to help",
"to make",
"to provide",
"to speak",
"to support",
"trade",
"turn to",
"understanding of",
"unemployment",
"united states",
"vendor performance",
"weekly hours",
"wholesale price index"]

In [17]:
trump_tweets['week'] = (trump_tweets.date - trump_tweets['date'].dt.weekday * np.timedelta64(1,'D'))
trump_tweets['week'] = pd.DatetimeIndex(trump_tweets['week']).normalize()
trump_tweets['day'] = pd.DatetimeIndex(trump_tweets['date']).normalize()
trump_tweets

,date,text,favorites,week,day
0,2021-06-07 22:14:15,Why does Fox News keep Chris Wallace? His rati...,1216,2021-06-07,2021-06-07
1,2021-06-07 22:14:06,Senator Lisa Murkowski has cost the great peop...,1171,2021-06-07,2021-06-07
2,2021-06-07 13:57:15,,2483,2021-06-07,2021-06-07
3,2021-06-04 18:46:45,Next time I’m in the White House there will be...,7720,2021-05-31,2021-06-04
4,2021-06-04 17:24:40,Facebook’s ruling is an insult to the record-s...,8062,2021-05-31,2021-06-04
...,...,...,...,...,...
43299,2016-01-01 06:08:06,"""""""@jallenaip: Hillary said she was in a """"Fog...",6008,2015-12-28,2016-01-01
43300,2016-01-01 06:07:28,Happy New Year from #MarALago! Thank you to my...,6860,2015-12-28,2016-01-01
2782,2016-01-01 05:18:23,#HappyNewYearAmerica! https://t.co/EeQb8PDrUe,7421,2015-12-28,2016-01-01
43301,2016-01-01 03:25:27,"HAPPY NEW YEAR &amp, THANK YOU! https://t.co/Y...",6978,2015-12-28,2016-01-01


In [18]:
# Count frequencies of economic keywords by week
countbyweek=defaultdict(int)
for index,row in trump_tweets.iterrows():
    for word in row['text'].split(' '):
#         print(word)
        if word in trade_matchers:
            countbyweek[row['week']] += 1
#     break
countbyweek

defaultdict(int,
            {Timestamp('2021-05-10 00:00:00'): 1,
             Timestamp('2021-04-19 00:00:00'): 1,
             Timestamp('2021-03-29 00:00:00'): 6,
             Timestamp('2021-03-15 00:00:00'): 2,
             Timestamp('2021-03-01 00:00:00'): 1,
             Timestamp('2021-02-22 00:00:00'): 1,
             Timestamp('2020-12-28 00:00:00'): 2,
             Timestamp('2020-12-14 00:00:00'): 1,
             Timestamp('2020-12-07 00:00:00'): 2,
             Timestamp('2020-11-16 00:00:00'): 2,
             Timestamp('2020-11-09 00:00:00'): 2,
             Timestamp('2020-11-02 00:00:00'): 7,
             Timestamp('2020-10-26 00:00:00'): 7,
             Timestamp('2020-10-19 00:00:00'): 3,
             Timestamp('2020-10-12 00:00:00'): 8,
             Timestamp('2020-10-05 00:00:00'): 1,
             Timestamp('2020-09-28 00:00:00'): 2,
             Timestamp('2020-09-21 00:00:00'): 2,
             Timestamp('2020-09-14 00:00:00'): 5,
             Timestamp('2020-09-0

In [19]:
# Count frequencies of economic keywords by day
countbyday=defaultdict(int)
for index,row in trump_tweets.iterrows():
    for word in row['text'].split(' '):
#         print(word)
        if word in trade_matchers:
            countbyday[row['day']] += 1
#     break
countbyday

defaultdict(int,
            {Timestamp('2021-05-12 00:00:00'): 1,
             Timestamp('2021-04-23 00:00:00'): 1,
             Timestamp('2021-03-31 00:00:00'): 5,
             Timestamp('2021-03-29 00:00:00'): 1,
             Timestamp('2021-03-20 00:00:00'): 2,
             Timestamp('2021-03-04 00:00:00'): 1,
             Timestamp('2021-02-22 00:00:00'): 1,
             Timestamp('2021-01-02 00:00:00'): 1,
             Timestamp('2020-12-29 00:00:00'): 1,
             Timestamp('2020-12-20 00:00:00'): 1,
             Timestamp('2020-12-11 00:00:00'): 1,
             Timestamp('2020-12-08 00:00:00'): 1,
             Timestamp('2020-11-22 00:00:00'): 1,
             Timestamp('2020-11-21 00:00:00'): 1,
             Timestamp('2020-11-15 00:00:00'): 1,
             Timestamp('2020-11-12 00:00:00'): 1,
             Timestamp('2020-11-07 00:00:00'): 1,
             Timestamp('2020-11-03 00:00:00'): 3,
             Timestamp('2020-11-02 00:00:00'): 3,
             Timestamp('2020-11-0

In [20]:
# Create a dataframe of the frequency counts of economic keywords by week
trump_weekdf = pd.DataFrame([countbyweek]).transpose().reset_index()
trump_weekdf.columns=(['Week','Count'])
trump_weekdf

,Week,Count
0,2021-05-10,1
1,2021-04-19,1
2,2021-03-29,6
3,2021-03-15,2
4,2021-03-01,1
...,...,...
221,2016-02-08,3
222,2016-01-18,3
223,2016-01-11,1
224,2016-01-04,2


In [21]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models 
trump_weekdf.to_csv('./Data/trump_weekdf_count.csv', index=False)

In [22]:
# Create a dataframe of the frequency counts of economic keywords by day
trump_daydf = pd.DataFrame([countbyday]).transpose().reset_index()
trump_daydf.columns=(['Date','Count'])
trump_daydf

,Date,Count
0,2021-05-12,1
1,2021-04-23,1
2,2021-03-31,5
3,2021-03-29,1
4,2021-03-20,2
...,...,...
536,2016-01-16,1
537,2016-01-07,1
538,2016-01-06,1
539,2016-01-03,1


In [23]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
trump_daydf.to_csv('./Data/trump_daydf_count.csv', index=False)

In [24]:
# Introduce negative financial keywords
Negative = ['abandon', 'abandoned', 'abandoning', 'abandonment', 'abandonments', 'abandons', 'abdicated',
                       'abdicates', 'abdicating', 'abdication', 'abdications', 'aberrant', 'aberration', 'aberrational',
                       'aberrations', 'abetting', 'abnormal', 'abnormalities', 'abnormality', 'abnormally', 'abolish',
                       'abolished', 'abolishes', 'abolishing', 'abrogate', 'abrogated', 'abrogates', 'abrogating',
                       'abrogation', 'abrogations', 'abrupt', 'abruptly', 'abruptness', 'absence', 'absences',
                       'absenteeism', 'abuse', 'abused', 'abuses', 'abusing', 'abusive', 'abusively', 'abusiveness',
                       'accident', 'accidental', 'accidentally', 'accidents', 'accusation', 'accusations', 'accuse',
                       'accused', 'accuses', 'accusing', 'acquiesce', 'acquiesced', 'acquiesces', 'acquiescing',
                       'acquit', 'acquits', 'acquittal', 'acquittals', 'acquitted', 'acquitting', 'adulterate',
                       'adulterated', 'adulterating', 'adulteration', 'adulterations', 'adversarial', 'adversaries',
                       'adversary', 'adverse', 'adversely', 'adversities', 'adversity', 'aftermath', 'aftermaths',
                       'against', 'aggravate', 'aggravated', 'aggravates', 'aggravating', 'aggravation', 'aggravations',
                       'alerted', 'alerting', 'alienate', 'alienated', 'alienates', 'alienating', 'alienation',
                       'alienations', 'allegation', 'allegations', 'allege', 'alleged', 'allegedly', 'alleges',
                       'alleging', 'annoy', 'annoyance', 'annoyances', 'annoyed', 'annoying', 'annoys', 'annul',
                       'annulled', 'annulling', 'annulment', 'annulments', 'annuls', 'anomalies', 'anomalous',
                       'anomalously', 'anomaly', 'anticompetitive', 'antitrust', 'argue', 'argued', 'arguing',
                       'argument', 'argumentative', 'arguments', 'arrearage', 'arrearages', 'arrears', 'arrest',
                       'arrested', 'arrests', 'artificially', 'assault', 'assaulted', 'assaulting', 'assaults',
                       'assertions', 'attrition', 'aversely', 'backdating', 'bad', 'bail', 'bailout', 'balk', 'balked',
                       'bankrupt', 'bankruptcies', 'bankruptcy', 'bankrupted', 'bankrupting', 'bankrupts', 'bans',
                       'barred', 'barrier', 'barriers', 'bottleneck', 'bottlenecks', 'boycott', 'boycotted',
                       'boycotting', 'boycotts', 'breach', 'breached', 'breaches', 'breaching', 'break', 'breakage',
                       'breakages', 'breakdown', 'breakdowns', 'breaking', 'breaks', 'bribe', 'bribed', 'briberies',
                       'bribery', 'bribes', 'bribing', 'bridge', 'broken', 'burden', 'burdened', 'burdening', 'burdens',
                       'burdensome', 'burned', 'calamities', 'calamitous', 'calamity', 'cancel', 'canceled',
                       'canceling', 'cancellation', 'cancellations', 'cancelled', 'cancelling', 'cancels', 'careless',
                       'carelessly', 'carelessness', 'catastrophe', 'catastrophes', 'catastrophic', 'catastrophically',
                       'caution', 'cautionary', 'cautioned', 'cautioning', 'cautions', 'cease', 'ceased', 'ceases',
                       'ceasing', 'censure', 'censured', 'censures', 'censuring', 'challenge', 'challenged',
                       'challenges', 'challenging', 'chargeoffs', 'circumvent', 'circumvented', 'circumventing',
                       'circumvention', 'circumventions', 'circumvents', 'claiming', 'claims', 'clawback', 'closed',
                       'closeout', 'closeouts', 'closing', 'closings', 'closure', 'closures', 'coerce', 'coerced',
                       'coerces', 'coercing', 'coercion', 'coercive', 'collapse', 'collapsed', 'collapses',
                       'collapsing', 'collision', 'collisions', 'collude', 'colluded', 'colludes', 'colluding',
                       'collusion', 'collusions', 'collusive', 'complain', 'complained', 'complaining', 'complains',
                       'complaint', 'complaints', 'complicate', 'complicated', 'complicates', 'complicating',
                       'complication', 'complications', 'compulsion', 'concealed', 'concealing', 'concede', 'conceded',
                       'concedes', 'conceding', 'concern', 'concerned', 'concerns', 'conciliating', 'conciliation',
                       'conciliations', 'condemn', 'condemnation', 'condemnations', 'condemned', 'condemning',
                       'condemns', 'condone', 'condoned', 'confess', 'confessed', 'confesses', 'confessing',
                       'confession', 'confine', 'confined', 'confinement', 'confinements', 'confines', 'confining',
                       'confiscate', 'confiscated', 'confiscates', 'confiscating', 'confiscation', 'confiscations',
                       'conflict', 'conflicted', 'conflicting', 'conflicts', 'confront', 'confrontation',
                       'confrontational', 'confrontations', 'confronted', 'confronting', 'confronts', 'confuse',
                       'confused', 'confuses', 'confusing', 'confusingly', 'confusion', 'conspiracies', 'conspiracy',
                       'conspirator', 'conspiratorial', 'conspirators', 'conspire', 'conspired', 'conspires',
                       'conspiring', 'contempt', 'contend', 'contended', 'contending', 'contends', 'contention',
                       'contentions', 'contentious', 'contentiously', 'contested', 'contesting', 'contraction',
                       'contractions', 'contradict', 'contradicted', 'contradicting', 'contradiction', 'contradictions',
                       'contradictory', 'contradicts', 'contrary', 'controversial', 'controversies', 'controversy',
                       'convict', 'convicted', 'convicting', 'conviction', 'convictions', 'corrected', 'correcting',
                       'correction', 'corrections', 'corrects', 'corrupt', 'corrupted', 'corrupting', 'corruption',
                       'corruptions', 'corruptly', 'corruptness', 'costly', 'counterclaim', 'counterclaimed',
                       'counterclaiming', 'counterclaims', 'counterfeit', 'counterfeited', 'counterfeiter',
                       'counterfeiters', 'counterfeiting', 'counterfeits', 'countermeasure', 'countermeasures', 'crime',
                       'crimes', 'criminal', 'criminally', 'criminals', 'crises', 'crisis', 'critical', 'critically',
                       'criticism', 'criticisms', 'criticize', 'criticized', 'criticizes', 'criticizing', 'crucial',
                       'crucially', 'culpability', 'culpable', 'culpably', 'cumbersome', 'curtail', 'curtailed',
                       'curtailing', 'curtailment', 'curtailments', 'curtails', 'cut', 'cutback', 'cutbacks',
                       'cyberattack', 'cyberattacks', 'cyberbullying', 'cybercrime', 'cybercrimes', 'cybercriminal',
                       'cybercriminals', 'damage', 'damaged', 'damages', 'damaging', 'dampen', 'dampened', 'danger',
                       'dangerous', 'dangerously', 'dangers', 'deadlock', 'deadlocked', 'deadlocking', 'deadlocks',
                       'deadweight', 'deadweights', 'debarment', 'debarments', 'debarred', 'deceased', 'deceit',
                       'deceitful', 'deceitfulness', 'deceive', 'deceived', 'deceives', 'deceiving', 'deception',
                       'deceptions', 'deceptive', 'deceptively', 'decline', 'declined', 'declines', 'declining',
                       'deface', 'defaced', 'defacement', 'defamation', 'defamations', 'defamatory', 'defame',
                       'defamed', 'defames', 'defaming', 'default', 'defaulted', 'defaulting', 'defaults', 'defeat',
                       'defeated', 'defeating', 'defeats', 'defect', 'defective', 'defects', 'defend', 'defendant',
                       'defendants', 'defended', 'defending', 'defends', 'defensive', 'defer', 'deficiencies',
                       'deficiency', 'deficient', 'deficit', 'deficits', 'defraud', 'defrauded', 'defrauding',
                       'defrauds', 'defunct', 'degradation', 'degradations', 'degrade', 'degraded', 'degrades',
                       'degrading', 'delay', 'delayed', 'delaying', 'delays', 'deleterious', 'deliberate',
                       'deliberated', 'deliberately', 'delinquencies', 'delinquency', 'delinquent', 'delinquently',
                       'delinquents', 'delist', 'delisted', 'delisting', 'delists', 'demise', 'demised', 'demises',
                       'demising', 'demolish', 'demolished', 'demolishes', 'demolishing', 'demolition', 'demolitions',
                       'demote', 'demoted', 'demotes', 'demoting', 'demotion', 'demotions', 'denial', 'denials',
                       'denied', 'denies', 'denigrate', 'denigrated', 'denigrates', 'denigrating', 'denigration',
                       'deny', 'denying', 'deplete', 'depleted', 'depletes', 'depleting', 'depletion', 'depletions',
                       'deprecation', 'depress', 'depressed', 'depresses', 'depressing', 'deprivation', 'deprive',
                       'deprived', 'deprives', 'depriving', 'derelict', 'dereliction', 'derogatory', 'destabilization',
                       'destabilize', 'destabilized', 'destabilizing', 'destroy', 'destroyed', 'destroying', 'destroys',
                       'destruction', 'destructive', 'detain', 'detained', 'detention', 'detentions', 'deter',
                       'deteriorate', 'deteriorated', 'deteriorates', 'deteriorating', 'deterioration',
                       'deteriorations', 'deterred', 'deterrence', 'deterrences', 'deterrent', 'deterrents',
                       'deterring', 'deters', 'detract', 'detracted', 'detracting', 'detriment', 'detrimental',
                       'detrimentally', 'detriments', 'devalue', 'devalued', 'devalues', 'devaluing', 'devastate',
                       'devastated', 'devastating', 'devastation', 'deviate', 'deviated', 'deviates', 'deviating',
                       'deviation', 'deviations', 'devolve', 'devolved', 'devolves', 'devolving', 'difficult',
                       'difficulties', 'difficultly', 'difficulty', 'diminish', 'diminished', 'diminishes',
                       'diminishing', 'diminution', 'disadvantage', 'disadvantaged', 'disadvantageous', 'disadvantages',
                       'disaffiliation', 'disagree', 'disagreeable', 'disagreed', 'disagreeing', 'disagreement',
                       'disagreements', 'disagrees', 'disallow', 'disallowance', 'disallowances', 'disallowed',
                       'disallowing', 'disallows', 'disappear', 'disappearance', 'disappearances', 'disappeared',
                       'disappearing', 'disappears', 'disappoint', 'disappointed', 'disappointing', 'disappointingly',
                       'disappointment', 'disappointments', 'disappoints', 'disapproval', 'disapprovals', 'disapprove',
                       'disapproved', 'disapproves', 'disapproving', 'disassociates', 'disassociating',
                       'disassociation', 'disassociations', 'disaster', 'disasters', 'disastrous', 'disastrously',
                       'disavow', 'disavowal', 'disavowed', 'disavowing', 'disavows', 'disciplinary', 'disclaim',
                       'disclaimed', 'disclaimer', 'disclaimers', 'disclaiming', 'disclaims', 'disclose', 'disclosed',
                       'discloses', 'disclosing', 'discontinuance', 'discontinuances', 'discontinuation',
                       'discontinuations', 'discontinue', 'discontinued', 'discontinues', 'discontinuing', 'discourage',
                       'discouraged', 'discourages', 'discouraging', 'discredit', 'discredited', 'discrediting',
                       'discredits', 'discrepancies', 'discrepancy', 'disfavor', 'disfavored', 'disfavoring',
                       'disfavors', 'disgorge', 'disgorged', 'disgorgement', 'disgorgements', 'disgorges', 'disgorging',
                       'disgrace', 'disgraceful', 'disgracefully', 'dishonest', 'dishonestly', 'dishonesty', 'dishonor',
                       'dishonorable', 'dishonorably', 'dishonored', 'dishonoring', 'dishonors', 'disincentives',
                       'disinterested', 'disinterestedly', 'disinterestedness', 'disloyal', 'disloyally', 'disloyalty',
                       'dismal', 'dismally', 'dismiss', 'dismissal', 'dismissals', 'dismissed', 'dismisses',
                       'dismissing', 'disorderly', 'disparage', 'disparaged', 'disparagement', 'disparagements',
                       'disparages', 'disparaging', 'disparagingly', 'disparities', 'disparity', 'displace',
                       'displaced', 'displacement', 'displacements', 'displaces', 'displacing', 'dispose', 'dispossess',
                       'dispossessed', 'dispossesses', 'dispossessing', 'disproportion', 'disproportional',
                       'disproportionate', 'disproportionately', 'dispute', 'disputed', 'disputes', 'disputing',
                       'disqualification', 'disqualifications', 'disqualified', 'disqualifies', 'disqualify',
                       'disqualifying', 'disregard', 'disregarded', 'disregarding', 'disregards', 'disreputable',
                       'disrepute', 'disrupt', 'disrupted', 'disrupting', 'disruption', 'disruptions', 'disruptive',
                       'disrupts', 'dissatisfaction', 'dissatisfied', 'dissent', 'dissented', 'dissenter', 'dissenters',
                       'dissenting', 'dissents', 'dissident', 'dissidents', 'dissolution', 'dissolutions', 'distort',
                       'distorted', 'distorting', 'distortion', 'distortions', 'distorts', 'distract', 'distracted',
                       'distracting', 'distraction', 'distractions', 'distracts', 'distress', 'distressed', 'disturb',
                       'disturbance', 'disturbances', 'disturbed', 'disturbing', 'disturbs', 'diversion', 'divert',
                       'diverted', 'diverting', 'diverts', 'divest', 'divested', 'divesting', 'divestiture',
                       'divestitures', 'divestment', 'divestments', 'divests', 'divorce', 'divorced', 'divulge',
                       'divulged', 'divulges', 'divulging', 'doubt', 'doubted', 'doubtful', 'doubts', 'downgrade',
                       'downgraded', 'downgrades', 'downgrading', 'downsize', 'downsized', 'downsizes', 'downsizing',
                       'downsizings', 'downtime', 'downtimes', 'downturn', 'downturns', 'downward', 'downwards', 'drag',
                       'drastic', 'drastically', 'drawback', 'drawbacks', 'dropped', 'drought', 'droughts', 'duress',
                       'dysfunction', 'dysfunctional', 'dysfunctions', 'easing', 'egregious', 'egregiously', 'embargo',
                       'embargoed', 'embargoes', 'embargoing', 'embarrass', 'embarrassed', 'embarrasses',
                       'embarrassing', 'embarrassment', 'embarrassments', 'embezzle', 'embezzled', 'embezzlement',
                       'embezzlements', 'embezzler', 'embezzles', 'embezzling', 'encroach', 'encroached', 'encroaches',
                       'encroaching', 'encroachment', 'encroachments', 'encumber', 'encumbered', 'encumbering',
                       'encumbers', 'encumbrance', 'encumbrances', 'endanger', 'endangered', 'endangering',
                       'endangerment', 'endangers', 'enjoin', 'enjoined', 'enjoining', 'enjoins', 'erode', 'eroded',
                       'erodes', 'eroding', 'erosion', 'erratic', 'erratically', 'erred', 'erring', 'erroneous',
                       'erroneously', 'error', 'errors', 'errs', 'escalate', 'escalated', 'escalates', 'escalating',
                       'evade', 'evaded', 'evades', 'evading', 'evasion', 'evasions', 'evasive', 'evict', 'evicted',
                       'evicting', 'eviction', 'evictions', 'evicts', 'exacerbate', 'exacerbated', 'exacerbates',
                       'exacerbating', 'exacerbation', 'exacerbations', 'exaggerate', 'exaggerated', 'exaggerates',
                       'exaggerating', 'exaggeration', 'excessive', 'excessively', 'exculpate', 'exculpated',
                       'exculpates', 'exculpating', 'exculpation', 'exculpations', 'exculpatory', 'exonerate',
                       'exonerated', 'exonerates', 'exonerating', 'exoneration', 'exonerations', 'exploit',
                       'exploitation', 'exploitations', 'exploitative', 'exploited', 'exploiting', 'exploits', 'expose',
                       'exposed', 'exposes', 'exposing', 'expropriate', 'expropriated', 'expropriates', 'expropriating',
                       'expropriation', 'expropriations', 'expulsion', 'expulsions', 'extenuating', 'fail', 'failed',
                       'failing', 'failings', 'fails', 'failure', 'failures', 'fallout', 'false', 'falsely',
                       'falsification', 'falsifications', 'falsified', 'falsifies', 'falsify', 'falsifying', 'falsity',
                       'fatalities', 'fatality', 'fatally', 'fault', 'faulted', 'faults', 'faulty', 'fear', 'fears',
                       'felonies', 'felonious', 'felony', 'fictitious', 'fined', 'fines', 'fired', 'firing', 'flaw',
                       'flawed', 'flaws', 'forbid', 'forbidden', 'forbidding', 'forbids', 'force', 'forced', 'forcing',
                       'foreclose', 'foreclosed', 'forecloses', 'foreclosing', 'foreclosure', 'foreclosures', 'forego',
                       'foregoes', 'foregone', 'forestall', 'forestalled', 'forestalling', 'forestalls', 'forfeit',
                       'forfeited', 'forfeiting', 'forfeits', 'forfeiture', 'forfeitures', 'forgers', 'forgery',
                       'fraud', 'frauds', 'fraudulence', 'fraudulent', 'fraudulently', 'frivolous', 'frivolously',
                       'frustrate', 'frustrated', 'frustrates', 'frustrating', 'frustratingly', 'frustration',
                       'frustrations', 'fugitive', 'fugitives', 'gratuitous', 'gratuitously', 'grievance', 'grievances',
                       'grossly', 'groundless', 'guilty', 'halt', 'halted', 'hamper', 'hampered', 'hampering',
                       'hampers', 'harass', 'harassed', 'harassing', 'harassment', 'hardship', 'hardships', 'harm',
                       'harmed', 'harmful', 'harmfully', 'harming', 'harms', 'harsh', 'harsher', 'harshest', 'harshly',
                       'harshness', 'hazard', 'hazardous', 'hazards', 'hinder', 'hindered', 'hindering', 'hinders',
                       'hindrance', 'hindrances', 'hostile', 'hostility', 'hurt', 'hurting', 'idle', 'idled', 'idling',
                       'ignore', 'ignored', 'ignores', 'ignoring', 'ill', 'illegal', 'illegalities', 'illegality',
                       'illegally', 'illegible', 'illicit', 'illicitly', 'illiquid', 'illiquidity', 'imbalance',
                       'imbalances', 'immature', 'immoral', 'impair', 'impaired', 'impairing', 'impairment',
                       'impairments', 'impairs', 'impasse', 'impasses', 'impede', 'impeded', 'impedes', 'impediment',
                       'impediments', 'impeding', 'impending', 'imperative', 'imperfection', 'imperfections', 'imperil',
                       'impermissible', 'implicate', 'implicated', 'implicates', 'implicating', 'impossibility',
                       'impossible', 'impound', 'impounded', 'impounding', 'impounds', 'impracticable', 'impractical',
                       'impracticalities', 'impracticality', 'imprisonment', 'improper', 'improperly', 'improprieties',
                       'impropriety', 'imprudent', 'imprudently', 'inability', 'inaccessible', 'inaccuracies',
                       'inaccuracy', 'inaccurate', 'inaccurately', 'inaction', 'inactions', 'inactivate', 'inactivated',
                       'inactivates', 'inactivating', 'inactivation', 'inactivations', 'inactivity', 'inadequacies',
                       'inadequacy', 'inadequate', 'inadequately', 'inadvertent', 'inadvertently', 'inadvisability',
                       'inadvisable', 'inappropriate', 'inappropriately', 'inattention', 'incapable', 'incapacitated',
                       'incapacity', 'incarcerate', 'incarcerated', 'incarcerates', 'incarcerating', 'incarceration',
                       'incarcerations', 'incidence', 'incidences', 'incident', 'incidents', 'incompatibilities',
                       'incompatibility', 'incompatible', 'incompetence', 'incompetency', 'incompetent',
                       'incompetently', 'incompetents', 'incomplete', 'incompletely', 'incompleteness', 'inconclusive',
                       'inconsistencies', 'inconsistency', 'inconsistent', 'inconsistently', 'inconvenience',
                       'inconveniences', 'inconvenient', 'incorrect', 'incorrectly', 'incorrectness', 'indecency',
                       'indecent', 'indefeasible', 'indefeasibly', 'indict', 'indictable', 'indicted', 'indicting',
                       'indictment', 'indictments', 'ineffective', 'ineffectively', 'ineffectiveness', 'inefficiencies',
                       'inefficiency', 'inefficient', 'inefficiently', 'ineligibility', 'ineligible', 'inequitable',
                       'inequitably', 'inequities', 'inequity', 'inevitable', 'inexperience', 'inexperienced',
                       'inferior', 'inflicted', 'infraction', 'infractions', 'infringe', 'infringed', 'infringement',
                       'infringements', 'infringes', 'infringing', 'inhibited', 'inimical', 'injunction', 'injunctions',
                       'injure', 'injured', 'injures', 'injuries', 'injuring', 'injurious', 'injury', 'inordinate',
                       'inordinately', 'inquiry', 'insecure', 'insensitive', 'insolvencies', 'insolvency', 'insolvent',
                       'instability', 'insubordination', 'insufficiency', 'insufficient', 'insufficiently',
                       'insurrection', 'insurrections', 'intentional', 'interfere', 'interfered', 'interference',
                       'interferences', 'interferes', 'interfering', 'intermittent', 'intermittently', 'interrupt',
                       'interrupted', 'interrupting', 'interruption', 'interruptions', 'interrupts', 'intimidation',
                       'intrusion', 'invalid', 'invalidate', 'invalidated', 'invalidates', 'invalidating',
                       'invalidation', 'invalidity', 'investigate', 'investigated', 'investigates', 'investigating',
                       'investigation', 'investigations', 'involuntarily', 'involuntary', 'irreconcilable',
                       'irreconcilably', 'irrecoverable', 'irrecoverably', 'irregular', 'irregularities',
                       'irregularity', 'irregularly', 'irreparable', 'irreparably', 'irreversible', 'jeopardize',
                       'jeopardized', 'justifiable', 'kickback', 'kickbacks', 'knowingly', 'lack', 'lacked', 'lacking',
                       'lackluster', 'lacks', 'lag', 'lagged', 'lagging', 'lags', 'lapse', 'lapsed', 'lapses',
                       'lapsing', 'late', 'laundering', 'layoff', 'layoffs', 'lie', 'limitation', 'limitations',
                       'lingering', 'liquidate', 'liquidated', 'liquidates', 'liquidating', 'liquidation',
                       'liquidations', 'liquidator', 'liquidators', 'litigant', 'litigants', 'litigate', 'litigated',
                       'litigates', 'litigating', 'litigation', 'litigations', 'lockout', 'lockouts', 'lose', 'loses',
                       'losing', 'loss', 'losses', 'lost', 'lying', 'malfeasance', 'malfunction', 'malfunctioned',
                       'malfunctioning', 'malfunctions', 'malice', 'malicious', 'maliciously', 'malpractice',
                       'manipulate', 'manipulated', 'manipulates', 'manipulating', 'manipulation', 'manipulations',
                       'manipulative', 'markdown', 'markdowns', 'misapplication', 'misapplications', 'misapplied',
                       'misapplies', 'misapply', 'misapplying', 'misappropriate', 'misappropriated', 'misappropriates',
                       'misappropriating', 'misappropriation', 'misappropriations', 'misbranded', 'miscalculate',
                       'miscalculated', 'miscalculates', 'miscalculating', 'miscalculation', 'miscalculations',
                       'mischaracterization', 'mischief', 'misclassification', 'misclassifications', 'misclassified',
                       'misclassify', 'miscommunication', 'misconduct', 'misdated', 'misdemeanor', 'misdemeanors',
                       'misdirected', 'mishandle', 'mishandled', 'mishandles', 'mishandling', 'misinform',
                       'misinformation', 'misinformed', 'misinforming', 'misinforms', 'misinterpret',
                       'misinterpretation', 'misinterpretations', 'misinterpreted', 'misinterpreting', 'misinterprets',
                       'misjudge', 'misjudged', 'misjudges', 'misjudging', 'misjudgment', 'misjudgments', 'mislabel',
                       'mislabeled', 'mislabeling', 'mislabelled', 'mislabels', 'mislead', 'misleading', 'misleadingly',
                       'misleads', 'misled', 'mismanage', 'mismanaged', 'mismanagement', 'mismanages', 'mismanaging',
                       'mismatch', 'mismatched', 'mismatches', 'mismatching', 'misplaced', 'misprice', 'mispricing',
                       'mispricings', 'misrepresent', 'misrepresentation', 'misrepresentations', 'misrepresented',
                       'misrepresenting', 'misrepresents', 'miss', 'missed', 'misses', 'misstate', 'misstated',
                       'misstatement', 'misstatements', 'misstates', 'misstating', 'misstep', 'missteps', 'mistake',
                       'mistaken', 'mistakenly', 'mistakes', 'mistaking', 'mistrial', 'mistrials', 'misunderstand',
                       'misunderstanding', 'misunderstandings', 'misunderstood', 'misuse', 'misused', 'misuses',
                       'misusing', 'monopolistic', 'monopolists', 'monopolization', 'monopolize', 'monopolized',
                       'monopolizes', 'monopolizing', 'monopoly', 'moratoria', 'moratorium', 'moratoriums',
                       'mothballed', 'mothballing', 'negative', 'negatively', 'negatives', 'neglect', 'neglected',
                       'neglectful', 'neglecting', 'neglects', 'negligence', 'negligences', 'negligent', 'negligently',
                       'nonattainment', 'noncompetitive', 'noncompliance', 'noncompliances', 'noncompliant',
                       'noncomplying', 'nonconforming', 'nonconformities', 'nonconformity', 'nondisclosure',
                       'nonfunctional', 'nonpayment', 'nonpayments', 'nonperformance', 'nonperformances',
                       'nonperforming', 'nonproducing', 'nonproductive', 'nonrecoverable', 'nonrenewal', 'nuisance',
                       'nuisances', 'nullification', 'nullifications', 'nullified', 'nullifies', 'nullify',
                       'nullifying', 'objected', 'objecting', 'objection', 'objectionable', 'objectionably',
                       'objections', 'obscene', 'obscenity', 'obsolescence', 'obsolete', 'obstacle', 'obstacles',
                       'obstruct', 'obstructed', 'obstructing', 'obstruction', 'obstructions', 'offence', 'offences',
                       'offend', 'offended', 'offender', 'offenders', 'offending', 'offends', 'omission', 'omissions',
                       'omit', 'omits', 'omitted', 'omitting', 'onerous', 'opportunistic', 'opportunistically',
                       'oppose', 'opposed', 'opposes', 'opposing', 'opposition', 'oppositions', 'outage', 'outages',
                       'outdated', 'outmoded', 'overage', 'overages', 'overbuild', 'overbuilding', 'overbuilds',
                       'overbuilt', 'overburden', 'overburdened', 'overburdening', 'overcapacities', 'overcapacity',
                       'overcharge', 'overcharged', 'overcharges', 'overcharging', 'overcome', 'overcomes',
                       'overcoming', 'overdue', 'overestimate', 'overestimated', 'overestimates', 'overestimating',
                       'overestimation', 'overestimations', 'overload', 'overloaded', 'overloading', 'overloads',
                       'overlook', 'overlooked', 'overlooking', 'overlooks', 'overpaid', 'overpayment', 'overpayments',
                       'overproduced', 'overproduces', 'overproducing', 'overproduction', 'overrun', 'overrunning',
                       'overruns', 'overshadow', 'overshadowed', 'overshadowing', 'overshadows', 'overstate',
                       'overstated', 'overstatement', 'overstatements', 'overstates', 'overstating', 'oversupplied',
                       'oversupplies', 'oversupply', 'oversupplying', 'overtly', 'overturn', 'overturned',
                       'overturning', 'overturns', 'overvalue', 'overvalued', 'overvaluing', 'panic', 'panics',
                       'penalize', 'penalized', 'penalizes', 'penalizing', 'penalties', 'penalty', 'peril', 'perils',
                       'perjury', 'perpetrate', 'perpetrated', 'perpetrates', 'perpetrating', 'perpetration', 'persist',
                       'persisted', 'persistence', 'persistent', 'persistently', 'persisting', 'persists', 'pervasive',
                       'pervasively', 'pervasiveness', 'petty', 'picket', 'picketed', 'picketing', 'plaintiff',
                       'plaintiffs', 'plea', 'plead', 'pleaded', 'pleading', 'pleadings', 'pleads', 'pleas', 'pled',
                       'poor', 'poorly', 'poses', 'posing', 'postpone', 'postponed', 'postponement', 'postponements',
                       'postpones', 'postponing', 'precipitated', 'precipitous', 'precipitously', 'preclude',
                       'precluded', 'precludes', 'precluding', 'predatory', 'prejudice', 'prejudiced', 'prejudices',
                       'prejudicial', 'prejudicing', 'premature', 'prematurely', 'pressing', 'pretrial', 'preventing',
                       'prevention', 'prevents', 'problem', 'problematic', 'problematical', 'problems', 'prolong',
                       'prolongation', 'prolongations', 'prolonged', 'prolonging', 'prolongs', 'prone', 'prosecute',
                       'prosecuted', 'prosecutes', 'prosecuting', 'prosecution', 'prosecutions', 'protest', 'protested',
                       'protester', 'protesters', 'protesting', 'protestor', 'protestors', 'protests', 'protracted',
                       'protraction', 'provoke', 'provoked', 'provokes', 'provoking', 'punished', 'punishes',
                       'punishing', 'punishment', 'punishments', 'punitive', 'purport', 'purported', 'purportedly',
                       'purporting', 'purports', 'question', 'questionable', 'questionably', 'questioned',
                       'questioning', 'questions', 'quit', 'quitting', 'racketeer', 'racketeering', 'rationalization',
                       'rationalizations', 'rationalize', 'rationalized', 'rationalizes', 'rationalizing',
                       'reassessment', 'reassessments', 'reassign', 'reassigned', 'reassigning', 'reassignment',
                       'reassignments', 'reassigns', 'recall', 'recalled', 'recalling', 'recalls', 'recession',
                       'recessionary', 'recessions', 'reckless', 'recklessly', 'recklessness', 'redact', 'redacted',
                       'redacting', 'redaction', 'redactions', 'redefault', 'redefaulted', 'redefaults', 'redress',
                       'redressed', 'redresses', 'redressing', 'refusal', 'refusals', 'refuse', 'refused', 'refuses',
                       'refusing', 'reject', 'rejected', 'rejecting', 'rejection', 'rejections', 'rejects',
                       'relinquish', 'relinquished', 'relinquishes', 'relinquishing', 'relinquishment',
                       'relinquishments', 'reluctance', 'reluctant', 'renegotiate', 'renegotiated', 'renegotiates',
                       'renegotiating', 'renegotiation', 'renegotiations', 'renounce', 'renounced', 'renouncement',
                       'renouncements', 'renounces', 'renouncing', 'reparation', 'reparations', 'repossessed',
                       'repossesses', 'repossessing', 'repossession', 'repossessions', 'repudiate', 'repudiated',
                       'repudiates', 'repudiating', 'repudiation', 'repudiations', 'resign', 'resignation',
                       'resignations', 'resigned', 'resigning', 'resigns', 'restate', 'restated', 'restatement',
                       'restatements', 'restates', 'restating', 'restructure', 'restructured', 'restructures',
                       'restructuring', 'restructurings', 'retaliate', 'retaliated', 'retaliates', 'retaliating',
                       'retaliation', 'retaliations', 'retaliatory', 'retribution', 'retributions', 'revocation',
                       'revocations', 'revoke', 'revoked', 'revokes', 'revoking', 'ridicule', 'ridiculed', 'ridicules',
                       'ridiculing', 'riskier', 'riskiest', 'risky', 'sabotage', 'sacrifice', 'sacrificed',
                       'sacrifices', 'sacrificial', 'sacrificing', 'scandalous', 'scandals', 'scrutinize',
                       'scrutinized', 'scrutinizes', 'scrutinizing', 'scrutiny', 'secrecy', 'seize', 'seized', 'seizes',
                       'seizing', 'sentenced', 'sentencing', 'serious', 'seriously', 'seriousness', 'setback',
                       'setbacks', 'sever', 'severe', 'severed', 'severely', 'severities', 'severity', 'sharply',
                       'shocked', 'shortage', 'shortages', 'shortfall', 'shortfalls', 'shrinkage', 'shrinkages', 'shut',
                       'shutdown', 'shutdowns', 'shuts', 'shutting', 'slander', 'slandered', 'slanderous', 'slanders',
                       'slippage', 'slippages', 'slow', 'slowdown', 'slowdowns', 'slowed', 'slower', 'slowest',
                       'slowing', 'slowly', 'slowness', 'sluggish', 'sluggishly', 'sluggishness', 'solvencies',
                       'solvency', 'spam', 'spammers', 'spamming', 'staggering', 'stagnant', 'stagnate', 'stagnated',
                       'stagnates', 'stagnating', 'stagnation', 'standstill', 'standstills', 'stolen', 'stoppage',
                       'stoppages', 'stopped', 'stopping', 'stops', 'strain', 'strained', 'straining', 'strains',
                       'stress', 'stressed', 'stresses', 'stressful', 'stressing', 'stringent', 'subjected',
                       'subjecting', 'subjection', 'subpoena', 'subpoenaed', 'subpoenas', 'substandard', 'sue', 'sued',
                       'sues', 'suffer', 'suffered', 'suffering', 'suffers', 'suing', 'summoned', 'summoning',
                       'summons', 'summonses', 'susceptibility', 'susceptible', 'suspect', 'suspected', 'suspects',
                       'suspend', 'suspended', 'suspending', 'suspends', 'suspension', 'suspensions', 'suspicion',
                       'suspicions', 'suspicious', 'suspiciously', 'taint', 'tainted', 'tainting', 'taints', 'tampered',
                       'tense', 'terminate', 'terminated', 'terminates', 'terminating', 'termination', 'terminations',
                       'testify', 'testifying', 'threat', 'threaten', 'threatened', 'threatening', 'threatens',
                       'threats', 'tightening', 'tolerate', 'tolerated', 'tolerates', 'tolerating', 'toleration',
                       'tortuous', 'tortuously', 'tragedies', 'tragedy', 'tragic', 'tragically', 'traumatic', 'trouble',
                       'troubled', 'troubles', 'turbulence', 'turmoil', 'unable', 'unacceptable', 'unacceptably',
                       'unaccounted', 'unannounced', 'unanticipated', 'unapproved', 'unattractive', 'unauthorized',
                       'unavailability', 'unavailable', 'unavoidable', 'unavoidably', 'unaware', 'uncollectable',
                       'uncollected', 'uncollectibility', 'uncollectible', 'uncollectibles', 'uncompetitive',
                       'uncompleted', 'unconscionable', 'unconscionably', 'uncontrollable', 'uncontrollably',
                       'uncontrolled', 'uncorrected', 'uncover', 'uncovered', 'uncovering', 'uncovers', 'undeliverable',
                       'undelivered', 'undercapitalized', 'undercut', 'undercuts', 'undercutting', 'underestimate',
                       'underestimated', 'underestimates', 'underestimating', 'underestimation', 'underfunded',
                       'underinsured', 'undermine', 'undermined', 'undermines', 'undermining', 'underpaid',
                       'underpayment', 'underpayments', 'underpays', 'underperform', 'underperformance',
                       'underperformed', 'underperforming', 'underperforms', 'underproduced', 'underproduction',
                       'underreporting', 'understate', 'understated', 'understatement', 'understatements',
                       'understates', 'understating', 'underutilization', 'underutilized', 'undesirable', 'undesired',
                       'undetected', 'undetermined', 'undisclosed', 'undocumented', 'undue', 'unduly', 'uneconomic',
                       'uneconomical', 'uneconomically', 'unemployed', 'unemployment', 'unethical', 'unethically',
                       'unexcused', 'unexpected', 'unexpectedly', 'unfair', 'unfairly', 'unfavorability', 'unfavorable',
                       'unfavorably', 'unfavourable', 'unfeasible', 'unfit', 'unfitness', 'unforeseeable', 'unforeseen',
                       'unforseen', 'unfortunate', 'unfortunately', 'unfounded', 'unfriendly', 'unfulfilled',
                       'unfunded', 'uninsured', 'unintended', 'unintentional', 'unintentionally', 'unjust',
                       'unjustifiable', 'unjustifiably', 'unjustified', 'unjustly', 'unknowing', 'unknowingly',
                       'unlawful', 'unlawfully', 'unlicensed', 'unliquidated', 'unmarketable', 'unmerchantable',
                       'unmeritorious', 'unnecessarily', 'unnecessary', 'unneeded', 'unobtainable', 'unoccupied',
                       'unpaid', 'unperformed', 'unplanned', 'unpopular', 'unpredictability', 'unpredictable',
                       'unpredictably', 'unpredicted', 'unproductive', 'unprofitability', 'unprofitable', 'unqualified',
                       'unrealistic', 'unreasonable', 'unreasonableness', 'unreasonably', 'unreceptive',
                       'unrecoverable', 'unrecovered', 'unreimbursed', 'unreliable', 'unremedied', 'unreported',
                       'unresolved', 'unrest', 'unsafe', 'unsalable', 'unsaleable', 'unsatisfactory', 'unsatisfied',
                       'unsavory', 'unscheduled', 'unsellable', 'unsold', 'unsound', 'unstabilized', 'unstable',
                       'unsubstantiated', 'unsuccessful', 'unsuccessfully', 'unsuitability', 'unsuitable', 'unsuitably',
                       'unsuited', 'unsure', 'unsuspected', 'unsuspecting', 'unsustainable', 'untenable', 'untimely',
                       'untrusted', 'untruth', 'untruthful', 'untruthfully', 'untruthfulness', 'untruths', 'unusable',
                       'unwanted', 'unwarranted', 'unwelcome', 'unwilling', 'unwillingness', 'upset', 'urgency',
                       'urgent', 'usurious', 'usurp', 'usurped', 'usurping', 'usurps', 'usury', 'vandalism', 'verdict',
                       'verdicts', 'vetoed', 'victims', 'violate', 'violated', 'violates', 'violating', 'violation',
                       'violations', 'violative', 'violator', 'violators', 'violence', 'violent', 'violently',
                       'vitiate', 'vitiated', 'vitiates', 'vitiating', 'vitiation', 'voided', 'voiding', 'volatile',
                       'volatility', 'vulnerabilities', 'vulnerability', 'vulnerable', 'vulnerably', 'warn', 'warned',
                       'warning', 'warnings', 'warns', 'wasted', 'wasteful', 'wasting', 'weak', 'weaken', 'weakened',
                       'weakening', 'weakens', 'weaker', 'weakest', 'weakly', 'weakness', 'weaknesses', 'willfully',
                       'worries', 'worry', 'worrying', 'worse', 'worsen', 'worsened', 'worsening', 'worsens', 'worst',
                       'worthless', 'writedown', 'writedowns', 'writeoff', 'writeoffs', 'wrong', 'wrongdoing',
                       'wrongdoings', 'wrongful', 'wrongfully', 'wrongly', 'negative', 'negatives', 'fail', 'fails', 'failing', 'failure', 'weak', 'weakness', 'weaknesses',
                      'difficult', 'difficulty', 'hurdle', 'hurdles', 'obstacle', 'obstacles', 'slump', 'slumps',
                      'slumping', 'slumped', 'uncertain', 'uncertainty', 'unsettled', 'unfavorable', 'downturn',
                      'depressed', 'disappoint', 'disappoints', 'disappointing', 'disappointed', 'disappointment',
                      'risk', 'risks', 'risky', 'threat', 'threats', 'penalty', 'penalties', 'down', 'decrease',
                      'decreases', 'decreasing', 'decreased', 'decline', 'declines', 'declining', 'declined', 'fall',
                      'falls', 'falling', 'fell', 'fallen', 'drop', 'drops', 'dropping', 'dropped', 'deteriorate',
                      'deteriorates', 'deteriorating', 'deteriorated', 'worsen', 'worsens', 'worsening', 'weaken',
                      'weakens', 'weakening', 'weakened', 'worse', 'worst', 'low', 'lower', 'lowest', 'less', 'least',
                      'smaller', 'smallest', 'shrink', 'shrinks', 'shrinking', 'shrunk', 'below', 'under', 'challenge',
                      'challenges', 'challenging', 'challenged'] 

In [25]:
# Introduce positive financial keywords
Positive = ['able', 'abundance', 'abundant', 'acclaimed', 'accomplish', 'accomplished', 'accomplishes',
                       'accomplishing', 'accomplishment', 'accomplishments', 'achieve', 'achieved', 'achievement',
                       'achievements', 'achieves', 'achieving', 'adequately', 'advancement', 'advancements', 'advances',
                       'advancing', 'advantage', 'advantaged', 'advantageous', 'advantageously', 'advantages',
                       'alliance', 'alliances', 'assure', 'assured', 'assures', 'assuring', 'attain', 'attained',
                       'attaining', 'attainment', 'attainments', 'attains', 'attractive', 'attractiveness', 'beautiful',
                       'beautifully', 'beneficial', 'beneficially', 'benefit', 'benefited', 'benefiting', 'benefitted',
                       'benefitting', 'best', 'better', 'bolstered', 'bolstering', 'bolsters', 'boom', 'booming',
                       'boost', 'boosted', 'breakthrough', 'breakthroughs', 'brilliant', 'charitable', 'collaborate',
                       'collaborated', 'collaborates', 'collaborating', 'collaboration', 'collaborations',
                       'collaborative', 'collaborator', 'collaborators', 'compliment', 'complimentary', 'complimented',
                       'complimenting', 'compliments', 'conclusive', 'conclusively', 'conducive', 'confident',
                       'constructive', 'constructively', 'courteous', 'creative', 'creatively', 'creativeness',
                       'creativity', 'delight', 'delighted', 'delightful', 'delightfully', 'delighting', 'delights',
                       'dependability', 'dependable', 'desirable', 'desired', 'despite', 'destined', 'diligent',
                       'diligently', 'distinction', 'distinctions', 'distinctive', 'distinctively', 'distinctiveness',
                       'dream', 'easier', 'easily', 'easy', 'effective', 'efficiencies', 'efficiency', 'efficient',
                       'efficiently', 'empower', 'empowered', 'empowering', 'empowers', 'enable', 'enabled', 'enables',
                       'enabling', 'encouraged', 'encouragement', 'encourages', 'encouraging', 'enhance', 'enhanced',
                       'enhancement', 'enhancements', 'enhances', 'enhancing', 'enjoy', 'enjoyable', 'enjoyably',
                       'enjoyed', 'enjoying', 'enjoyment', 'enjoys', 'enthusiasm', 'enthusiastic', 'enthusiastically',
                       'excellence', 'excellent', 'excelling', 'excels', 'exceptional', 'exceptionally', 'excited',
                       'excitement', 'exciting', 'exclusive', 'exclusively', 'exclusiveness', 'exclusives',
                       'exclusivity', 'exemplary', 'fantastic', 'favorable', 'favorably', 'favored', 'favoring',
                       'favorite', 'favorites', 'friendly', 'gain', 'gained', 'gaining', 'gains', 'good', 'great',
                       'greater', 'greatest', 'greatly', 'greatness', 'happiest', 'happily', 'happiness', 'happy',
                       'highest', 'honor', 'honorable', 'honored', 'honoring', 'honors', 'ideal', 'impress',
                       'impressed', 'impresses', 'impressing', 'impressive', 'impressively', 'improve', 'improved',
                       'improvement', 'improvements', 'improves', 'improving', 'incredible', 'incredibly',
                       'influential', 'informative', 'ingenuity', 'innovate', 'innovated', 'innovates', 'innovating',
                       'innovation', 'innovations', 'innovative', 'innovativeness', 'innovator', 'innovators',
                       'insightful', 'inspiration', 'inspirational', 'integrity', 'invent', 'invented', 'inventing',
                       'invention', 'inventions', 'inventive', 'inventiveness', 'inventor', 'inventors', 'leadership',
                       'leading', 'loyal', 'lucrative', 'meritorious', 'opportunities', 'opportunity', 'optimistic',
                       'outperform', 'outperformed', 'outperforming', 'outperforms', 'perfect', 'perfected',
                       'perfectly', 'perfects', 'pleasant', 'pleasantly', 'pleased', 'pleasure', 'plentiful', 'popular',
                       'popularity', 'positive', 'positively', 'preeminence', 'preeminent', 'premier', 'premiere',
                       'prestige', 'prestigious', 'proactive', 'proactively', 'proficiency', 'proficient',
                       'proficiently', 'profitability', 'profitable', 'profitably', 'progress', 'progressed',
                       'progresses', 'progressing', 'prospered', 'prospering', 'prosperity', 'prosperous', 'prospers',
                       'rebound', 'rebounded', 'rebounding', 'receptive', 'regain', 'regained', 'regaining', 'resolve',
                       'revolutionize', 'revolutionized', 'revolutionizes', 'revolutionizing', 'reward', 'rewarded',
                       'rewarding', 'rewards', 'satisfaction', 'satisfactorily', 'satisfactory', 'satisfied',
                       'satisfies', 'satisfy', 'satisfying', 'smooth', 'smoothing', 'smoothly', 'smooths', 'solves',
                       'solving', 'spectacular', 'spectacularly', 'stability', 'stabilization', 'stabilizations',
                       'stabilize', 'stabilized', 'stabilizes', 'stabilizing', 'stable', 'strength', 'strengthen',
                       'strengthened', 'strengthening', 'strengthens', 'strengths', 'strong', 'stronger', 'strongest',
                       'succeed', 'succeeded', 'succeeding', 'succeeds', 'success', 'successes', 'successful',
                       'successfully', 'superior', 'surpass', 'surpassed', 'surpasses', 'surpassing', 'transparency',
                       'tremendous', 'tremendously', 'unmatched', 'unparalleled', 'unsurpassed', 'upturn', 'upturns',
                       'valuable', 'versatile', 'versatility', 'vibrancy', 'vibrant', 'win', 'winner', 'winners',
                       'winning', 'worthy', 'positive', 'positives', 'success', 'successes', 'successful', 'succeed', 'succeeds',
                      'succeeding', 'succeeded', 'accomplish', 'accomplishes', 'accomplishing', 'accomplished',
                      'accomplishment', 'accomplishments', 'strong', 'strength', 'strengths', 'certain', 'certainty',
                      'definite', 'solid', 'excellent', 'good', 'leading', 'achieve', 'achieves', 'achieved',
                      'achieving', 'achievement', 'achievements', 'progress', 'progressing', 'deliver', 'delivers',
                      'delivered', 'delivering', 'leader', 'leading', 'pleased', 'reward', 'rewards', 'rewarding',
                      'rewarded', 'opportunity', 'opportunities', 'enjoy', 'enjoys', 'enjoying', 'enjoyed',
                      'encouraged', 'encouraging', 'up', 'increase', 'increases', 'increasing', 'increased', 'rise',
                      'rises', 'rising', 'rose', 'risen', 'improve', 'improves', 'improving', 'improved', 'improvement',
                      'improvements', 'strengthen', 'strengthens', 'strengthening', 'strengthened', 'stronger',
                      'strongest', 'better', 'best', 'more', 'most', 'above', 'record', 'high', 'higher', 'highest',
                      'greater', 'greatest', 'larger', 'largest', 'grow', 'grows', 'growing', 'grew', 'grown', 'growth',
                      'expand', 'expands', 'expanding', 'expanded', 'expansion', 'exceed', 'exceeds', 'exceeded',
                      'exceeding', 'beat', 'beats', 'beating'] 

In [26]:
# Sentiment analysis using the above negative and positive keywords.
sentbydaypos=defaultdict(int)
sentbydayneg=defaultdict(int)
sentbyday=defaultdict(int)
for index,row in trump_tweets.iterrows():
    for word in row['text'].split(' '):
#         print(word)
        if word in Positive:
            sentbydaypos[row['day']] += 1
            sentbyday[row['day']] += 1
        if word in Negative:
            sentbydayneg[row['day']] += 1
            sentbyday[row['day']] =  sentbyday[row['day']] -1
            
#     break

In [27]:
# positive sentiment counts of frequency of positive keywords by day
sentbydaypos

defaultdict(int,
            {Timestamp('2021-06-07 00:00:00'): 4,
             Timestamp('2021-06-04 00:00:00'): 5,
             Timestamp('2021-06-03 00:00:00'): 3,
             Timestamp('2021-06-02 00:00:00'): 9,
             Timestamp('2021-06-01 00:00:00'): 4,
             Timestamp('2021-05-31 00:00:00'): 3,
             Timestamp('2021-05-27 00:00:00'): 6,
             Timestamp('2021-05-26 00:00:00'): 6,
             Timestamp('2021-05-25 00:00:00'): 9,
             Timestamp('2021-05-24 00:00:00'): 2,
             Timestamp('2021-05-22 00:00:00'): 5,
             Timestamp('2021-05-21 00:00:00'): 2,
             Timestamp('2021-05-20 00:00:00'): 3,
             Timestamp('2021-05-19 00:00:00'): 17,
             Timestamp('2021-05-18 00:00:00'): 3,
             Timestamp('2021-05-16 00:00:00'): 2,
             Timestamp('2021-05-15 00:00:00'): 14,
             Timestamp('2021-05-14 00:00:00'): 7,
             Timestamp('2021-05-13 00:00:00'): 3,
             Timestamp('2021-05

In [28]:
# Create a dataframe of the positive sentiment counts of frequency of positive keywords by day
trump_sentbydayposdf = pd.DataFrame([sentbydaypos]).transpose().reset_index()
trump_sentbydayposdf.columns=(['Date','Pos_Count'])
trump_sentbydayposdf

,Date,Pos_Count
0,2021-06-07,4
1,2021-06-04,5
2,2021-06-03,3
3,2021-06-02,9
4,2021-06-01,4
...,...,...
1722,2016-01-05,3
1723,2016-01-04,2
1724,2016-01-03,4
1725,2016-01-02,3


In [29]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
trump_sentbydayposdf.to_csv('./Data/trump_sentbydayposdf_count.csv', index=False)

In [30]:
# Negative sentiment counts of frequency of negative keywords by day
sentbydayneg

defaultdict(int,
            {Timestamp('2021-06-07 00:00:00'): 4,
             Timestamp('2021-06-04 00:00:00'): 3,
             Timestamp('2021-06-03 00:00:00'): 8,
             Timestamp('2021-06-02 00:00:00'): 8,
             Timestamp('2021-05-31 00:00:00'): 2,
             Timestamp('2021-05-27 00:00:00'): 3,
             Timestamp('2021-05-26 00:00:00'): 6,
             Timestamp('2021-05-25 00:00:00'): 6,
             Timestamp('2021-05-24 00:00:00'): 5,
             Timestamp('2021-05-22 00:00:00'): 2,
             Timestamp('2021-05-20 00:00:00'): 1,
             Timestamp('2021-05-19 00:00:00'): 35,
             Timestamp('2021-05-15 00:00:00'): 21,
             Timestamp('2021-05-13 00:00:00'): 4,
             Timestamp('2021-05-12 00:00:00'): 22,
             Timestamp('2021-05-11 00:00:00'): 19,
             Timestamp('2021-05-10 00:00:00'): 2,
             Timestamp('2021-05-07 00:00:00'): 9,
             Timestamp('2021-05-06 00:00:00'): 3,
             Timestamp('2021-

In [31]:
# Create a dataframe of the negative sentiment counts of frequency of negative keywords by day
trump_sentbydaynegdf = pd.DataFrame([sentbydayneg]).transpose().reset_index()
trump_sentbydaynegdf.columns=(['Date','Neg_Count'])
trump_sentbydaynegdf

,Date,Neg_Count
0,2021-06-07,4
1,2021-06-04,3
2,2021-06-03,8
3,2021-06-02,8
4,2021-05-31,2
...,...,...
1634,2016-01-07,9
1635,2016-01-05,2
1636,2016-01-03,5
1637,2016-01-02,4


In [32]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
trump_sentbydaynegdf.to_csv('./Data/trump_sentbydaynegdf_count.csv', index=False)

In [33]:
# (positive - negative) sentiment counts of frequency of (positive - negative) keywords by day
sentbyday

defaultdict(int,
            {Timestamp('2021-06-07 00:00:00'): 0,
             Timestamp('2021-06-04 00:00:00'): 2,
             Timestamp('2021-06-03 00:00:00'): -5,
             Timestamp('2021-06-02 00:00:00'): 1,
             Timestamp('2021-06-01 00:00:00'): 4,
             Timestamp('2021-05-31 00:00:00'): 1,
             Timestamp('2021-05-27 00:00:00'): 3,
             Timestamp('2021-05-26 00:00:00'): 0,
             Timestamp('2021-05-25 00:00:00'): 3,
             Timestamp('2021-05-24 00:00:00'): -3,
             Timestamp('2021-05-22 00:00:00'): 3,
             Timestamp('2021-05-21 00:00:00'): 2,
             Timestamp('2021-05-20 00:00:00'): 2,
             Timestamp('2021-05-19 00:00:00'): -18,
             Timestamp('2021-05-18 00:00:00'): 3,
             Timestamp('2021-05-16 00:00:00'): 2,
             Timestamp('2021-05-15 00:00:00'): -7,
             Timestamp('2021-05-14 00:00:00'): 7,
             Timestamp('2021-05-13 00:00:00'): -1,
             Timestamp('202

In [34]:
# Create a dataframe of the (positive - negative) sentiment counts of frequency of (positive - negative) keywords by day
trump_sentbydaydf = pd.DataFrame([sentbyday]).transpose().reset_index()
trump_sentbydaydf.columns=(['Date','Sent_Count'])
trump_sentbydaydf

,Date,Sent_Count
0,2021-06-07,0
1,2021-06-04,2
2,2021-06-03,-5
3,2021-06-02,1
4,2021-06-01,4
...,...,...
1809,2016-01-05,1
1810,2016-01-04,2
1811,2016-01-03,-1
1812,2016-01-02,-1


In [35]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
trump_sentbydaydf.to_csv('./Data/trump_sentbydaydf_count.csv', index=False)

In [36]:
# Sentiment analysis using the above negative and positive keywords.
sentbyweekpos=defaultdict(int)
sentbyweekneg=defaultdict(int)
sentbyweek=defaultdict(int)
for index,row in trump_tweets.iterrows():
    for word in row['text'].split(' '):
#         print(word)
        if word in Positive:
            sentbyweekpos[row['week']] += 1
            sentbyweek[row['week']] += 1
        if word in Negative:
            sentbyweekneg[row['week']] += 1
            sentbyweek[row['week']] =  sentbyweek[row['week']] -1
            
#     break

In [37]:
# positive sentiment counts of frequency of positive keywords by week
sentbyweekpos

defaultdict(int,
            {Timestamp('2021-06-07 00:00:00'): 4,
             Timestamp('2021-05-31 00:00:00'): 24,
             Timestamp('2021-05-24 00:00:00'): 23,
             Timestamp('2021-05-17 00:00:00'): 30,
             Timestamp('2021-05-10 00:00:00'): 52,
             Timestamp('2021-05-03 00:00:00'): 15,
             Timestamp('2021-04-26 00:00:00'): 10,
             Timestamp('2021-04-19 00:00:00'): 13,
             Timestamp('2021-04-12 00:00:00'): 17,
             Timestamp('2021-04-05 00:00:00'): 27,
             Timestamp('2021-03-29 00:00:00'): 26,
             Timestamp('2021-03-15 00:00:00'): 13,
             Timestamp('2021-03-08 00:00:00'): 7,
             Timestamp('2021-03-01 00:00:00'): 34,
             Timestamp('2021-02-22 00:00:00'): 15,
             Timestamp('2021-01-04 00:00:00'): 14,
             Timestamp('2020-12-28 00:00:00'): 62,
             Timestamp('2020-12-21 00:00:00'): 37,
             Timestamp('2020-12-14 00:00:00'): 46,
             Tim

In [38]:
# Create a dataframe of the positive sentiment counts of frequency of positive keywords by week
trump_sentbyweekposdf = pd.DataFrame([sentbydaypos]).transpose().reset_index()
trump_sentbyweekposdf.columns=(['Week','Pos_Count'])
trump_sentbyweekposdf

,Week,Pos_Count
0,2021-06-07,4
1,2021-06-04,5
2,2021-06-03,3
3,2021-06-02,9
4,2021-06-01,4
...,...,...
1722,2016-01-05,3
1723,2016-01-04,2
1724,2016-01-03,4
1725,2016-01-02,3


In [39]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
trump_sentbyweekposdf.to_csv('./Data/trump_sentbyweekposdf_count.csv', index=False)

In [40]:
# Negative sentiment counts of frequency of negative keywords by day
sentbyweekneg

defaultdict(int,
            {Timestamp('2021-06-07 00:00:00'): 4,
             Timestamp('2021-05-31 00:00:00'): 21,
             Timestamp('2021-05-24 00:00:00'): 20,
             Timestamp('2021-05-17 00:00:00'): 38,
             Timestamp('2021-05-10 00:00:00'): 68,
             Timestamp('2021-05-03 00:00:00'): 24,
             Timestamp('2021-04-26 00:00:00'): 17,
             Timestamp('2021-04-19 00:00:00'): 8,
             Timestamp('2021-04-12 00:00:00'): 20,
             Timestamp('2021-04-05 00:00:00'): 33,
             Timestamp('2021-03-29 00:00:00'): 28,
             Timestamp('2021-03-22 00:00:00'): 2,
             Timestamp('2021-03-15 00:00:00'): 26,
             Timestamp('2021-03-08 00:00:00'): 4,
             Timestamp('2021-03-01 00:00:00'): 39,
             Timestamp('2021-02-22 00:00:00'): 9,
             Timestamp('2021-01-04 00:00:00'): 28,
             Timestamp('2020-12-28 00:00:00'): 67,
             Timestamp('2020-12-21 00:00:00'): 51,
             Timest

In [41]:
# Create a dataframe of the positive sentiment counts of frequency of positive keywords by week
trump_sentbyweeknegdf = pd.DataFrame([sentbydaypos]).transpose().reset_index()
trump_sentbyweeknegdf.columns=(['Week','Neg_Count'])
trump_sentbyweeknegdf

,Week,Neg_Count
0,2021-06-07,4
1,2021-06-04,5
2,2021-06-03,3
3,2021-06-02,9
4,2021-06-01,4
...,...,...
1722,2016-01-05,3
1723,2016-01-04,2
1724,2016-01-03,4
1725,2016-01-02,3


In [42]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
trump_sentbyweeknegdf.to_csv('./Data/trump_sentbyweeknegdf_count.csv', index=False)

In [43]:
# (positive - negative) sentiment counts of frequency of (positive - negative) keywords by week
sentbyweek

defaultdict(int,
            {Timestamp('2021-06-07 00:00:00'): 0,
             Timestamp('2021-05-31 00:00:00'): 3,
             Timestamp('2021-05-24 00:00:00'): 3,
             Timestamp('2021-05-17 00:00:00'): -8,
             Timestamp('2021-05-10 00:00:00'): -16,
             Timestamp('2021-05-03 00:00:00'): -9,
             Timestamp('2021-04-26 00:00:00'): -7,
             Timestamp('2021-04-19 00:00:00'): 5,
             Timestamp('2021-04-12 00:00:00'): -3,
             Timestamp('2021-04-05 00:00:00'): -6,
             Timestamp('2021-03-29 00:00:00'): -2,
             Timestamp('2021-03-22 00:00:00'): -2,
             Timestamp('2021-03-15 00:00:00'): -13,
             Timestamp('2021-03-08 00:00:00'): 3,
             Timestamp('2021-03-01 00:00:00'): -5,
             Timestamp('2021-02-22 00:00:00'): 6,
             Timestamp('2021-01-04 00:00:00'): -14,
             Timestamp('2020-12-28 00:00:00'): -5,
             Timestamp('2020-12-21 00:00:00'): -14,
             Tim

In [44]:
# Create a dataframe of the (positive - negative) sentiment counts of frequency of (positive - negative) keywords by week
trump_sentbyweekdf = pd.DataFrame([sentbyweek]).transpose().reset_index()
trump_sentbyweekdf.columns=(['Week','Sent_Count'])
trump_sentbyweekdf

,Week,Sent_Count
0,2021-06-07,0
1,2021-05-31,3
2,2021-05-24,3
3,2021-05-17,-8
4,2021-05-10,-16
...,...,...
274,2016-01-25,8
275,2016-01-18,4
276,2016-01-11,5
277,2016-01-04,9


In [45]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
trump_sentbyweekdf.to_csv('./Data/trump_sentbyweekdf_count.csv', index=False)

In [46]:
# Count numbers of favorites by week
trump_favw = trump_tweets.loc[:, ["week", "favorites"]]
trump_favw = trump_favw.groupby(['week']).sum()
trump_favw

,favorites
week,
2015-12-28,243732
2016-01-04,349430
2016-01-11,435835
2016-01-18,492822
2016-01-25,761643
...,...
2021-05-10,209231
2021-05-17,76257
2021-05-24,91641


In [47]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
trump_favw.to_csv('./Data/trump_favw_count.csv', index=True)

In [48]:
# Count numbers of favorites by day
trump_favd = trump_tweets.loc[:, ["day", "favorites"]]
trump_favd = trump_favd.groupby(['day']).sum()
trump_favd.rename(columns={"day": "date"}, inplace=True)
trump_favd

,favorites
day,
2016-01-01,68955
2016-01-02,73713
2016-01-03,101064
2016-01-04,20981
2016-01-05,47671
...,...
2021-06-01,4141
2021-06-02,15034
2021-06-03,17598


In [49]:
# Save the dataframe in a csv file for uses as a possible feature in predictive machine learning models
trump_favd.to_csv('./Data/trump_favd_count.csv', index=True)